### Variables: <br>
<b>Mutation: mu</b><br>
Default value: 0.00<br>
The chance of mutation, from 0 to 1. <br>

<b>Correlation: r</b><br>
Default value: 0.00<br>
The chance of correlation, from 0 to 1. <br>

<b>Number of Runs: runs</b><br>
Default value: 10<br>
The number of times the program runs a randomized simulation according to the provided matrix.<br>
The program will average results from all runs to arrive at a percentage spread of strategies. <br>

<b>Maximum Number of Generations: maxgen</b><br>
Default value: 1000<br>
The maximum number of replications in a given simulation.<br>
This number will not be reached by most traditional games.<br>

<b>Print Runs: printruns</b><br>
Default value: True<br>
A boolean (True or False) that determines if the simulation shows results for each individual run in addition to the averaged results.<br>

<b>Print Generations: printgen</b><br>
Default value: True<br>
A boolean (True or False) that determines if the simulation shows how many generations it took for the simulation to stabilize.<br>

<b>Random Matrix: randfill</b><br>
Default value: True<br>
A boolean (True or False) that determines whether the matrix has random or uniform distribution.<br>

<b>Starting Proportions: consistent_proportions</b><br>
Default value: None<br>
An array of probabilities that add up to one of the same size as the number of strategies in your population, such as [0.3, 0.7]. Only works if randfill is false.<br>

<b>Population Interaction Level: interactions</b><br>
Default value: 0.5<br>
The level of interaction between two populations in run_two_pops, from 0 (no interaction) to 1 (perfect interaction). 

In [93]:
#----------------------------------------------EDIT WITH CAUTION--------------------------------------------------

import numpy as np
import time

# Fill the matrix with random proportions
def rand_fill(TYPES):
    x = 0
    while x == 0:
        random = np.random.rand(TYPES)
        y = 0 - np.log(random)
        x = np.sum(y)
    pop = y / x
    return pop


# Fill the matrix with equal proportions
def consistent_fill(TYPES, proportions=None):
    if proportions is not None and len(proportions) == TYPES:
        return np.array(proportions)
    else:
        return np.full(TYPES, 1.0 / TYPES)


# Represents one generation in the population
def replicate(pop, oldpop, TYPES, matrix, mu, r):
    oldpop = np.copy(pop)
    fitness = np.zeros(TYPES)
    
    # For each strategy, calculate its payoff
    for i in range(TYPES):
        for j in range(TYPES):
            payoff_ij = matrix[i][j] * pop[j]
            fitness[i] += r * (matrix[i][i]) + (1 - r) * payoff_ij
    
    # Find the average fitness
    avefit = np.sum(fitness * pop)
    
    # For each strategy, change its proportion according to its payoff
    for i in range(TYPES):
        pop[i] = pop[i] * ((1 - mu) * fitness[i] / avefit) + mu / TYPES
    
    return pop, oldpop


# Represents a single generation in an environment which may have multiple populations
def replicate_two_pops(pop1, pop2, TYPES, matrix, mu, r, interaction):
    oldpop1 = np.copy(pop1)
    oldpop2 = np.copy(pop2)
    fitness1 = np.zeros(TYPES)
    fitness2 = np.zeros(TYPES)

    # Calculate payoffs for population 1
    for i in range(TYPES):
        for j in range(TYPES):
            # Intra-group interactions
            intra_payoff = (1 - interaction) * matrix[i][j] * pop1[j]
            # Inter-group interactions
            inter_payoff = interaction * matrix[i][j] * pop2[j]
            # Set payoffs of pop 1
            fitness1[i] += r * matrix[i][i] + intra_payoff + inter_payoff

    # Calculate payoffs for population 2 in a similar manner
    for i in range(TYPES):
        for j in range(TYPES):
            # Intra-group interactions
            intra_payoff = (1 - interaction) * matrix[i][j] * pop2[j]
            # Inter-group interactions
            inter_payoff = interaction * matrix[i][j] * pop1[j]
            # Set payoffs of pop 2
            fitness2[i] += r * matrix[i][i] + intra_payoff + inter_payoff


    # Adjust proportions based on fitness
    avefit1 = np.sum(fitness1 * pop1)
    for i in range(TYPES):
        pop1[i] = pop1[i] * ((1 - mu) * fitness1[i] / avefit1) + mu / TYPES

    # Adjust proportions for population 2 in a similar manner
    avefit2 = np.sum(fitness2 * pop2)
    for i in range(TYPES):
        pop2[i] = pop2[i] * ((1 - mu) * fitness2[i] / avefit2) + mu / TYPES

    return pop1, oldpop1, pop2, oldpop2


# Detect if the population is stable
def detect_stable(oldpop, pop, threshold=1e-5):
    return np.all(np.abs(oldpop - pop) < threshold)


# Print an individual run
def print_run(pop, gen, totals, printgen):
    formatted_proportions = ["{:.3f}".format(prop) if prop >= 0.001 else "_____" for prop in pop]
    if printgen:
        print(" ".join(formatted_proportions), "Generations:", gen)
    else:
            print(" ".join(formatted_proportions))
    totals += pop
    return totals


# Print the averaged results for each strategy
def print_results(sum_proportions, RUNS):
    mean_proportions = sum_proportions / RUNS
    formatted_proportions = ["{:.3f}".format(prop) if prop >= 0.0001 else "0.00" for prop in mean_proportions]
    print("\033[1m\nMean Proportions Across Simulations: \033[0m")
    print(" ".join(formatted_proportions))


# Print the averaged results for each strategy
def print_results_two_pops(sum_proportions1, sum_proportions2, runs):
    TYPES = len(sum_proportions1)
    print("\033[1mPopulation 1:\033[0m")
    for t in range(TYPES):
        print(f"Type {t + 1}: {sum_proportions1[t] / runs:.4f}")
    print("\033[1mPopulation 2:\033[0m")
    for t in range(TYPES):
        print(f"Type {t + 1}: {sum_proportions2[t] / runs:.4f}")

# Run the program
def run(matrix, mu=0.00, r=0.00, runs=10, maxgen=1000, printruns=True, 
        printgen=False, randfill=True, proportions=None):
    TYPES=matrix.shape[0]
    np.random.seed(int(time.time()))
    
    totals = np.zeros(TYPES)
    sum_proportions = np.zeros(TYPES)
    
    # For each run, do this
    for i in range(runs):
        pop = rand_fill(TYPES) if (randfill) else consistent_fill(TYPES, proportions)
        oldpop = np.zeros(TYPES)
        gen = 0
        stable = False
        
        while not stable and gen < maxgen:
            pop, oldpop = replicate(pop, oldpop, TYPES, matrix, mu, r)
            stable = detect_stable(oldpop, pop)
            gen += 1
            
        if printruns:
            totals = print_run(pop, gen, totals, printgen)
        
        sum_proportions += pop
        
    print_results(sum_proportions, runs)
    
# Run the program with two populations
def run_two_pops(matrix, mu=0.00, r=0.00, runs=10, maxgen=1000, printruns=True, 
                 printgen=False, randfill=True, interaction=0.5, proportions=None):
    TYPES = matrix.shape[0]
    np.random.seed(int(time.time()))

    totals1 = np.zeros(TYPES)
    totals2 = np.zeros(TYPES)
    sum_proportions1 = np.zeros(TYPES)
    sum_proportions2 = np.zeros(TYPES)

    for i in range(runs):
        # Initialize populations
        pop1 = rand_fill(TYPES) if randfill else consistent_fill(TYPES, proportions)
        pop2 = rand_fill(TYPES) if randfill else consistent_fill(TYPES, proportions)

        oldpop1 = np.zeros(TYPES)
        oldpop2 = np.zeros(TYPES)
        
        gen = 0
        stable1 = False
        stable2 = False

        while (not stable1 or not stable2) and gen < maxgen:
            pop1, oldpop1, pop2, oldpop2 = replicate_two_pops(pop1, pop2, TYPES, matrix, mu, r, interaction)

            stable1 = detect_stable(oldpop1, pop1)
            stable2 = detect_stable(oldpop2, pop2)

            gen += 1

        if printruns:
            totals1 = print_run_two_pops("Population 1: ", pop1, gen, totals1, printgen)
            totals2 = print_run_two_pops("Population 2: ", pop2, gen, totals2, printgen)
            print("\n")
        
        sum_proportions1 += pop1
        sum_proportions2 += pop2

    print("\033[1m\nFinal Mean Proportions Across Simulations: \033[0m")
    print_results_two_pops(sum_proportions1, sum_proportions2, runs)

In [94]:
#----------------------------------------------RUNNING SIMULATIONS------------------------------------------------
"""
Payoff Matrices
Symmetric games only - payoffs listed are only for player 1
"""
# Stag Hunt
staghunt = np.array([
    [4, 0],
    [3, 3]])

# Prisoner's Dilemma
prisonersdilemma = np.array([
    [2, 0],
    [3, 1]])

# Dancing Game
dancinggame = np.array([
    [0, 1],
    [1, 0]])

# Prisoners Dilemma with Punishment - Cnp (cooperate with no punish), Dnp (defect with no punish), 
# E (cooperate then punish defect), A (coin flip then punish)
punishdilemma0 = np.array([
    [2, 0, 2, -1],
    [3, 1, 1, 0],
    [2, -1, 2, -1.5],
    [1.5, -0.5, 0.5, -1.5]])

punishdilemma = np.array([
    [12, 10, 12, 9],
    [13, 11, 11, 10],
    [12, 9, 12, 8.5],
    [11.5, 9.5, 10.5, 8.5]])

fullpunishdilemma = np.array([
    [2, 2, 0, 0, 0, 0, -2, -2], #Cnn (staunch utilitarian)
    [2, 2, 0, 0, -1, -1, -3, -3], #Cnp (enforcer)
    [1, 1, -1, -1, 0, 0, -2, -2], #Cpn (hates himself)
    [1, 1, -1, -1, -1, -1, -3, -3], #Cpp (not very intelliget)
    [3, 1, 3, 1, 1, -1, 1, -1], #Dnn (game theorist)
    [3, 1, 1, 1, 0, -2, 0, -2], #Dnp (hates himself)
    [2, 0, 2, 0, 1, -1, 1, -1], #Dpn (doesn't like people who don't know game theory)
    [2, 0, 2, 0, 0, -2, 0, -2]]) #Dpp (minimize opponent's payoff)

# Reduced Ultimatum Game
smallultimatum = np.array([
    [5.0, 2.5, 6.0, 5.0],
    [2.5, 5.0, 5.0, 0.0],
    [4.0, 5.0, 5.0, 1.5],
    [5.0, 0.0, 3.5, 5.0]])

# Mini Ultimatum Game
miniultimatum = np.array([
    [5.0, 0.5, 7.0, 2.5],
    [4.5, 0.0, 7.0, 2.5],
    [3.0, 3.0, 5.0, 5.0],
    [2.5, 2.5, 5.0, 5.0]])

# 7-3 Ultimatum Game
fullultimatum = np.array([
    [5.0, 5.0, 3.5, 1.5, 6.0, 6.0, 2.5, 2.5],
    [5.0, 5.0, 1.5, 0.0, 3.5, 3.5, 0.0, 0.0],
    [1.5, 3.5, 0.0, 0.0, 6.0, 6.0, 2.5, 2.5], 
    [3.5, 0.0, 0.0, 0.0, 3.5, 3.5, 0.0, 0.0],
    [4.0, 1.5, 4.0, 1.5, 5.0, 2.5, 5.0, 2.5], 
    [4.0, 1.5, 4.0, 1.5, 2.5, 0.0, 5.0, 0.0],
    [2.5, 0.0, 2.5, 0.0, 5.0, 5.0, 5.0, 2.5],
    [2.5, 0.0, 2.5, 0.0, 2.5, 0.0, 2.5, 0.0]])

# 9-1 Ultimatum Game
skyrmsfullultimatum = np.array([
    [10, 1, 1, 10, 14, 5, 5, 14],
    [9, 0, 0, 9, 9, 0, 0, 9],
    [9, 0, 0, 9, 14, 5, 5, 14],
    [10, 1, 1, 10, 9, 0, 0, 9],
    [6, 1, 6, 1, 10, 5, 10, 1],
	[5, 0, 5, 0, 5, 0, 5, 0],
	[5, 0, 5, 0, 10, 5, 10, 5],
	[6, 1, 6, 1, 5, 0, 5, 0]])

In [96]:
# Change a value x by setting it to a value e.g. main(matrix, r=0.5, printruns=False)
# Alterable values: mu, r, runs, maxgen, printruns, printgen, randfill, consistent_proportions, interaction
run_two_pops(dancinggame, printgen=False, interaction=1)

Population 1:  _____ 1.000
Population 2:  1.000 _____


Population 1:  1.000 _____
Population 2:  _____ 1.000


Population 1:  _____ 1.000
Population 2:  1.000 _____


Population 1:  1.000 _____
Population 2:  _____ 1.000


Population 1:  _____ 1.000
Population 2:  1.000 _____


Population 1:  _____ 1.000
Population 2:  1.000 _____


Population 1:  1.000 _____
Population 2:  _____ 1.000


Population 1:  1.000 _____
Population 2:  _____ 1.000


Population 1:  _____ 1.000
Population 2:  1.000 _____


Population 1:  1.000 _____
Population 2:  _____ 1.000



Final Mean Proportions Across Simulations: 
Population 1:
Type 1: 0.5000
Type 2: 0.5000
Population 2:
Type 1: 0.5000
Type 2: 0.5000
